[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/show1d/show1d_all_features.ipynb)

# Show1D — All Features

Comprehensive demo of the interactive 1D viewer: spectra, profiles, convergence curves,
multi-trace overlay, axis calibration, log scale, grid, legend, mutation methods,
and state persistence.

In [1]:
try:
    import google.colab
    !pip install -q -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ quantem-widget
except ImportError:
    pass  # Not in Colab, skip

In [2]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

env: ANYWIDGET_HMR=1


In [3]:
import numpy as np
from pathlib import Path
from quantem.widget import Show1D


def make_eels_spectrum(n=512, seed=0):
    """Simulate EELS spectrum with zero-loss peak, plasmon, and core-loss edges."""
    rng = np.random.default_rng(seed)
    energy = np.linspace(-20, 800, n)
    zlp = 1000 * np.exp(-0.5 * (energy / 3) ** 2)
    plasmon = 200 * np.exp(-0.5 * ((energy - 15) / 5) ** 2)
    plasmon += 80 * np.exp(-0.5 * ((energy - 30) / 8) ** 2)
    bg = np.where(energy > 5, 5000 * (energy.clip(5, None) / 5) ** -2.5, 0)
    edge_onset = 532
    edge = np.where(energy > edge_onset, 15 * ((energy - edge_onset).clip(0, None) / 50) ** 0.4 * np.exp(-(energy - edge_onset) / 200), 0)
    spec = zlp + plasmon + bg + edge + rng.poisson(2, n).astype(np.float32)
    return energy.astype(np.float32), spec.astype(np.float32)


def make_edx_spectrum(n=1024, seed=0):
    """Simulate EDX spectrum with characteristic X-ray peaks on Bremsstrahlung background."""
    rng = np.random.default_rng(seed)
    energy = np.linspace(0, 20, n)  # keV
    # Bremsstrahlung background (Kramers)
    bg = 500 * np.exp(-0.3 * energy) * (energy + 0.1)
    # Characteristic peaks: Si-K (1.74), Ti-K (4.51), Fe-K (6.40), Cu-K (8.04), Zn-K (8.63)
    peaks = [(1.74, 800, 0.06), (4.51, 400, 0.08), (6.40, 600, 0.08), (8.04, 350, 0.09), (8.63, 200, 0.09)]
    spec = bg.copy()
    for center, amp, sigma in peaks:
        spec += amp * np.exp(-0.5 * ((energy - center) / sigma) ** 2)
    spec = np.maximum(spec, 0) + rng.poisson(3, n).astype(np.float32)
    return energy.astype(np.float32), spec.astype(np.float32)


def make_radial_profile(n=256, seed=0):
    """Simulate radial intensity profile from a diffraction pattern."""
    rng = np.random.default_rng(seed)
    q = np.linspace(0, 15, n)  # 1/nm
    # Amorphous ring + crystalline peaks
    amorphous = 50 * np.exp(-0.5 * ((q - 3.5) / 1.5) ** 2)
    peaks = [(2.0, 100, 0.15), (3.46, 60, 0.15), (4.0, 80, 0.15), (5.3, 40, 0.2), (6.93, 30, 0.2)]
    profile = amorphous
    for center, amp, sigma in peaks:
        profile += amp * np.exp(-0.5 * ((q - center) / sigma) ** 2)
    profile += 5 * rng.standard_normal(n)
    return q.astype(np.float32), np.maximum(profile, 0).astype(np.float32)


def make_convergence_curve(n=200, seed=0):
    """Simulate optimization convergence (e.g. ptychography reconstruction)."""
    rng = np.random.default_rng(seed)
    epochs = np.arange(n, dtype=np.float32)
    loss = 10.0 * np.exp(-0.02 * epochs) + 0.1 + 0.05 * rng.standard_normal(n)
    return epochs, loss.astype(np.float32)


def make_beam_current(n=500, seed=0):
    """Simulate beam current log over time (e.g. during TEM session)."""
    rng = np.random.default_rng(seed)
    t = np.linspace(0, 60, n)  # minutes
    # Slow drift + stepwise filament adjustment
    current = 120 - 0.3 * t + 5 * np.sin(0.1 * t)
    step = np.where(t > 30, 8, 0)  # filament boost at 30 min
    current = current + step + 2 * rng.standard_normal(n)
    return t.astype(np.float32), current.astype(np.float32)


print("Generator functions ready.")

Generator functions ready.


/Users/macbook/miniforge3/envs/bob-env/lib/python3.11/site-packages/anywidget/_util.py:283: UserWarning: anywidget: Live-reloading feature is disabled. To enable, please install the 'watchfiles' package.
  start_thread=_should_start_thread(path),


---
## Single EELS Spectrum

Basic usage with calibrated energy axis and axis labels.

In [4]:
energy, spec = make_eels_spectrum()
Show1D(spec, x=energy, title="EELS — O-K Edge", x_label="Energy Loss", x_unit="eV", y_label="Counts")

Show1D(512 points)

## EDX Spectrum

Energy-dispersive X-ray spectrum with characteristic peaks.

In [5]:
edx_energy, edx_spec = make_edx_spectrum()
Show1D(edx_spec, x=edx_energy, title="EDX Spectrum", x_label="Energy", x_unit="keV", y_label="Counts")

Show1D(1024 points)

## Radial Profile from Diffraction

Azimuthally averaged diffraction intensity vs. scattering vector.

In [6]:
q, profile = make_radial_profile()
Show1D(profile, x=q, title="Radial Profile", x_label="q", x_unit="1/nm", y_label="Intensity")

Show1D(256 points)

## Multi-Trace Overlay — EELS Comparison

Compare spectra from different sample regions with distinct colors and legend.

In [7]:
specs = [make_eels_spectrum(seed=i)[1] for i in range(4)]
Show1D(
    specs,
    x=energy,
    labels=["Grain Interior", "Grain Boundary", "Precipitate", "Matrix"],
    title="EELS — Spatial Comparison",
    x_label="Energy Loss",
    x_unit="eV",
    y_label="Counts",
)

Show1D(4 traces × 512 points)

## Log Scale — Convergence Curves

Compare different reconstruction algorithms on a log-scale Y axis.

In [8]:
epochs, loss1 = make_convergence_curve(seed=0)
_, loss2 = make_convergence_curve(seed=1)
_, loss3 = make_convergence_curve(seed=2)

# Make curves converge at different rates
loss2 = loss2 * 1.5
loss3 = loss3 * 0.7

Show1D(
    [loss1, loss2, loss3],
    x=epochs,
    labels=["ePIE", "rPIE", "ML-PIE"],
    title="Reconstruction Convergence",
    x_label="Iteration",
    y_label="Error",
    log_scale=True,
)

Show1D(3 traces × 200 points)

## Custom Colors and Line Width

In [9]:
Show1D(
    [loss1, loss2],
    x=epochs,
    labels=["Adam", "SGD"],
    colors=["#e91e63", "#00bcd4"],
    title="Custom Colors",
    x_label="Epoch",
    y_label="Loss",
    line_width=2.5,
)

Show1D(2 traces × 200 points)

## Beam Current Log

Monitor beam current drift over a TEM session.

In [10]:
t, current = make_beam_current()
Show1D(current, x=t, title="Beam Current", x_label="Time", x_unit="min", y_label="Current", y_unit="pA")

Show1D(500 points)

## Hide Controls & Stats

Minimal view for publication or embedding.

In [11]:
Show1D(
    spec,
    x=energy,
    title="Clean View",
    x_label="Energy Loss",
    x_unit="eV",
    show_controls=False,
    show_stats=False,
    show_grid=False,
)

Show1D(512 points)

## Simple Array (No X Axis)

When no X values are provided, indices are used.

In [12]:
Show1D(np.sin(np.linspace(0, 4 * np.pi, 300)).astype(np.float32), title="Sine Wave")

Show1D(300 points)

## 2D Array Input — Multiple Traces from Matrix

Pass a 2D array where each row is a trace.

In [13]:
x = np.linspace(0, 2 * np.pi, 200)
traces = np.stack([np.sin(x + phase).astype(np.float32) for phase in np.linspace(0, np.pi, 5)])
Show1D(traces, x=x.astype(np.float32), title="Phase-Shifted Sinusoids", x_label="Angle", x_unit="rad")

Show1D(5 traces × 200 points)

---
## Mutation — `set_data()`

Replace data while preserving display settings.

In [14]:
w = Show1D(spec, x=energy, title="Mutable Plot", x_label="Energy", log_scale=True)
w

Show1D(512 points)

In [ ]:
# Replace with EDX data — log_scale and title are preserved
w.set_data(edx_spec, x=edx_energy)

## Mutation — `add_trace()` / `remove_trace()`

In [ ]:
w2 = Show1D(spec, x=energy, title="Add/Remove Demo")
w2

In [ ]:
# Add traces one at a time
_, spec2 = make_eels_spectrum(seed=5)
_, spec3 = make_eels_spectrum(seed=10)
w2.add_trace(spec2, label="Region B")
w2.add_trace(spec3, label="Region C", color="#ff5722")

In [ ]:
# Remove the first trace
w2.remove_trace(0)

---
## State Persistence

Save and restore display settings across sessions.

In [ ]:
w_save = Show1D(
    spec,
    x=energy,
    title="Saved State Demo",
    x_label="Energy Loss",
    x_unit="eV",
    log_scale=True,
    show_grid=False,
    line_width=2.0,
)
w_save.summary()

In [ ]:
# Save to file
w_save.save("show1d_state.json")

# Inspect state_dict
import json
print(json.dumps(w_save.state_dict(), indent=2))

In [ ]:
# Restore from file — all display settings are preserved
w_loaded = Show1D(spec, x=energy, state="show1d_state.json")
w_loaded.summary()

In [ ]:
# Cleanup
Path("show1d_state.json").unlink(missing_ok=True)

## Repr

In [ ]:
print(repr(Show1D(np.ones(100, dtype=np.float32))))
print(repr(Show1D(np.ones((3, 50), dtype=np.float32))))